In [1]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from emotions_utils import *

d:\diploma\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
CORE_PATH = "../"
ORIG_PATH = f"{CORE_PATH}/emotions"
SAVE_LOGS_PATH = f"{CORE_PATH}/missclassified/new_dataset/"
SAVE_MODELS_PATH = f"{CORE_PATH}/models/experiment_with_new_dataset"

In [3]:
data = ImageFolder(ORIG_PATH)

In [4]:
import pickle

with open("test_indices.pickle", "rb") as file:
    test_indices = pickle.load(file)
print(len(test_indices))

test_data = torch.utils.data.Subset(data, test_indices)
orig_dataset = EmotionsDataset(
    test_data,
    transforms=inf_transforms
)

123


In [5]:
class_to_idx = data.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

class_to_idx

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'sadness': 5,
 'wonder': 6}

# AffectNetDataset

In [6]:
train = ImageFolder("../AffectNetDataset")

train

Dataset ImageFolder
    Number of datapoints: 23916
    Root location: ../AffectNetDataset

In [7]:
train.class_to_idx

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'sadness': 5,
 'wonder': 6}

In [8]:
Counter(train.targets)

Counter({4: 5044, 6: 4039, 0: 3218, 3: 3176, 5: 3091, 1: 2871, 2: 2477})

In [9]:
torch.manual_seed(0)

train_size = 0.75
test_size = 1 - train_size

train_data, test_data = torch.utils.data.random_split(
    train, [train_size, test_size]
)

In [10]:
train_dataset = EmotionsDataset(train_data, train_transforms)
test_dataset = EmotionsDataset(test_data, inf_transforms)

In [11]:
BATCH_SIZE = 16
LR = 0.001
EPOCHS = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

In [12]:
train_loader = get_loader(train_dataset, BATCH_SIZE)
test_loader = DataLoader(test_dataset, BATCH_SIZE)

In [13]:
import timm

model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=len(class_to_idx))
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [103]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_effnetb4_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

100%|██████████| 1122/1122 [03:26<00:00,  5.44it/s]


EPOCH: 1, train_loss: 1.3938110597942985, val_loss: 1.029560433939002
train_f1: 0.4604500891265597, val_f1: 0.5991279776373359



100%|██████████| 1122/1122 [03:14<00:00,  5.77it/s]


EPOCH: 2, train_loss: 1.1520176940308025, val_loss: 1.036764813199213
train_f1: 0.5607731729055259, val_f1: 0.6228579241614001



100%|██████████| 1122/1122 [03:19<00:00,  5.64it/s]


...

EPOCH: 12, train_loss: 0.7563105015649161, val_loss: 1.020820849993333
train_f1: 0.7204768270944741, val_f1: 0.6632231404958678



100%|██████████| 1122/1122 [03:11<00:00,  5.86it/s]


EPOCH: 13, train_loss: 0.7519575080654725, val_loss: 1.0437844484760364
train_f1: 0.7174688057040999, val_f1: 0.6698924404472534



100%|██████████| 1122/1122 [03:10<00:00,  5.88it/s]


EPOCH: 14, train_loss: 0.7130709737215299, val_loss: 1.1483665214451804
train_f1: 0.734792780748663, val_f1: 0.6538496596985902

Early Stopping!


In [107]:
torch.save(model.state_dict(), f"{SAVE_MODELS_PATH}/effnetb4_15_train.pt")
shutil.copyfile("best_effnetb4_15.pt", f"{SAVE_MODELS_PATH}/best_effnetb4_15.pt")

'..//models/experiment_with_new_dataset/best_effnetb4_15.pt'

In [14]:
best =  timm.create_model('efficientnet_b4', pretrained=True, num_classes=len(class_to_idx))
best.load_state_dict(torch.load(f"{SAVE_MODELS_PATH}/best_effnetb4_15.pt"))
best.eval()
best = best.to(device)

In [109]:
test_model(model, test_loader, device)

0.6538496596985902


In [19]:
test_model(best, test_loader, device)

0.6759236752552261


In [26]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [28]:
get_metrics_report(actual, pred)

{'Accuracy': 0.5934959349593496,
 'Precision_macro': 0.6263734584742988,
 'Precision_micro': 0.5934959349593496,
 'Recall_macro': 0.5938919726889651,
 'Recall_micro': 0.5934959349593496,
 'ROC_AUC': {0: 0.7490170380078637,
  1: 0.6219635627530364,
  2: 0.7778761061946904,
  3: 0.7667832167832168,
  4: 0.8674757281553398,
  5: 0.74,
  6: 0.8190819081908192}}

In [30]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 8/14

contempt emotion
Overall images: 19
Correctly predicted 5/19

disgust emotion
Overall images: 10
Correctly predicted 6/10

fear emotion
Overall images: 13
Correctly predicted 8/13

joy emotion
Overall images: 20
Correctly predicted 18/20

sadness emotion
Overall images: 25
Correctly predicted 12/25

wonder emotion
Overall images: 22
Correctly predicted 16/22



In [ ]:
with open("test_indices.pickle", "rb") as file:
    test_indices = pickle.load(file)

inf_images = [data.imgs[x][0] for x in test_indices]
get_mistaken_images_report(inf_images, actual, pred, "effnetb4_15", idx_to_class, SAVE_LOGS_PATH)